Libraries Integration

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
import tensorflow.keras.layers as layers
import sklearn.metrics as accuracy



Loading training data


In [4]:
labels_file = pd.read_csv("ArSL_Data_Labels.csv")
labels_file.head()

,#,File_Name,Class
0,1,AIN (1).JPG,ain
1,2,AIN (10).JPG,ain
2,3,AIN (100).jpg,ain
3,4,AIN (1000).JPG,ain
4,5,AIN (1001).JPG,ain


In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
labels = encoder.fit_transform(labels_file['Class'])   # ['ain','ba',...] → [0,1,...]

df = tf.keras.utils.image_dataset_from_directory(
    "ArASL_Database_54K_Final",
    labels=labels.tolist(),
    label_mode='int',
    color_mode="grayscale",
    image_size=(64, 64),
    batch_size=32
)

num_classes = len(encoder.classes_)

Found 54049 files belonging to 32 classes.


In [6]:
size = df.cardinality().numpy()
train_size = int(0.8*size)
test_size = size - train_size
train_set = df.take(train_size)
test_set = df.skip(train_size)

In [7]:
normalization = layers.Rescaling(1./255,input_shape = (64,64,1))
def normalize(x,y):
    return normalization(x), y
train_set = train_set.map(normalize)
test_set = test_set.map(normalize)


c:\Users\YOUSSEF\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
###Accuracy: 94%
model = tf.keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])
'''
###Accuracy: 50%
model1 = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(units = 128,activation = 'relu'),
    layers.Dense(units = 64,activation = 'relu'),
    layers.Dense(units = num_classes,activation = 'softmax')
])
'''

"\n###Accuracy: 50%\nmodel1 = tf.keras.Sequential([\n    layers.Flatten(),\n    layers.Dense(units = 128,activation = 'relu'),\n    layers.Dense(units = 64,activation = 'relu'),\n    layers.Dense(units = num_classes,activation = 'softmax')\n])\n"

In [9]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [10]:
model.fit(train_set,epochs = 20)

Epoch 1/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 123s 89ms/step - accuracy: 0.4584 - loss: 1.7711
Epoch 2/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 67s 50ms/step - accuracy: 0.7807 - loss: 0.6570
Epoch 3/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 65s 48ms/step - accuracy: 0.8511 - loss: 0.4519
Epoch 4/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 62s 45ms/step - accuracy: 0.8819 - loss: 0.3565
Epoch 5/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 62s 46ms/step - accuracy: 0.9042 - loss: 0.2914
Epoch 6/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 63s 47ms/step - accuracy: 0.9189 - loss: 0.2483
Epoch 7/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 66s 49ms/step - accuracy: 0.9272 - loss: 0.2177
Epoch 8/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 62s 46ms/step - accuracy: 0.9345 - loss: 0.1947
Epoch 9/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 63s 47ms/step - accuracy: 0.9414 - loss: 0.1753
Epoch 10/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 66s 48ms/step - accuracy: 0.9444 - loss: 0.1636
Epoch 11/20
1352/1352 ━━━━━━━━━━━━━━━━━━━━ 61s 45ms/step - accuracy: 0.9500 - loss: 0.1493
Epoch 1

In [13]:
model.metrics_names
model.evaluate(test_set)


338/338 ━━━━━━━━━━━━━━━━━━━━ 30s 61ms/step - accuracy: 0.9735 - loss: 0.1263


[0.12628726661205292, 0.9734817147254944]

In [14]:
model.save('main_model.h5')


In [20]:
image = tf.keras.utils.load_img("AIN (1).JPG",
                                color_mode='grayscale',target_size=(64,64))
input_arr = tf.keras.utils.img_to_array(image)
predicted_array = model.predict(np.array([input_arr/255]))
print(np.argmax(predicted_array))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
0
